In [1]:
print("he")

he


In [2]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time

/tmp/ipykernel_226586/1691537879.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Преобразование обучающих данных для расширения обучающей выборки и её нормализация
# Для валидационной (тестовой) выборки только нормализация
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(244),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(244),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



full_dataset = datasets.ImageFolder(root='train/simpsons_dataset')
total_size = len(full_dataset)
val_size = int(0.2 * total_size)  # let's assume 20% for validation
train_size = total_size - val_size

# Split the dataset into train and validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']



# специальный класс для загрузки данных в виде батчей
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=2)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

In [1]:
! pip install numba 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 24.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 86.3 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [1]:
import torch

# Custom BCE Loss Implementation
def custom_bce_loss(y_real, y_pred):
    return (y_pred - y_real * y_pred + torch.log(1 + torch.exp(-y_pred))).mean()

# Test data
y_real = torch.tensor([1, 0, 1, 0], dtype=torch.float32)
y_pred = torch.tensor([3.0, -2.0, 1.0, 0.0], dtype=torch.float32)  # Logits

# Calculate custom BCE Loss
custom_loss = custom_bce_loss(y_real, y_pred)
print("Custom BCE Loss:", custom_loss.item())

# PyTorch built-in BCEWithLogitsLoss
bce_loss = torch.nn.BCEWithLogitsLoss()
torch_loss = bce_loss(y_pred, y_real)
print("Torch BCEWithLogitsLoss:", torch_loss.item())


Custom BCE Loss: 0.2954810559749603
Torch BCEWithLogitsLoss: 0.2954810857772827
